<a href="https://colab.research.google.com/github/hassanfv/hfvSPH_on_Unified_Memory_GPU/blob/main/hfv_sph_um_gpu_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile hfv_sph_gpu.cu
#include <iostream>
#include <fstream>
#include <cmath>
#include <string>
#include <vector>
#include <sstream>
#include "myCppSPHLibs.h"
using namespace std;


float max_finder(float *arr, int N){

      float max_val = 0.0;
      for(int i = 0; i < N; i++){
        if(arr[i] >= max_val){
          max_val = arr[i];
        }
      }
      return max_val;
    }

    float min_finder(float *arr, int N){

      float min_val = arr[0];
      for(int i = 0; i < N; i++){
        if(arr[i] <= min_val){
          min_val = arr[i];
        }
      }
      return min_val;
    }


int main(){

  // Reading params.GPU file.
  ifstream infile;
  infile.open("params.GPU");

  int N;
  float c_0, gammah, Rcld_in_pc, Rcld_in_cm, Mcld_in_g, muu, Mach;
  float grav_const_in_cgs, G;

  if (infile.is_open()){
    infile >> N;
    infile >> c_0;
    infile >> gammah;
    infile >> Rcld_in_pc;
    infile >> Rcld_in_cm;
    infile >> Mcld_in_g;
    infile >> muu;
    infile >> Mach;
    infile >> grav_const_in_cgs;
    infile >> G;
    infile.close();
  }else{
    cout << "File Not Found !!!" << endl;
  }

  float UnitRadius_in_cm = Rcld_in_cm;
  float UnitRadius_in_cm_2 = UnitRadius_in_cm*UnitRadius_in_cm;

  float UnitMass_in_g = Mcld_in_g;
  //--------------------- To avoid getting inf ! --------
  float UnitDensity_in_cgs = UnitMass_in_g / UnitRadius_in_cm_2;
  UnitDensity_in_cgs = UnitDensity_in_cgs / UnitRadius_in_cm;
  //-------------------------
  float Unit_u_in_cgs = grav_const_in_cgs * UnitMass_in_g / UnitRadius_in_cm;
  float Unit_P_in_cgs = UnitDensity_in_cgs * Unit_u_in_cgs;
  float unitVelocity = sqrt(grav_const_in_cgs * UnitMass_in_g / UnitRadius_in_cm);

  // Reading Hydra file.
  string fname = "GPU_IC_Anathpin_150k_RAND.csv"; //!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  vector<vector<string>> content;
  vector<string> row;
  string line, word;
  
  fstream file (fname, ios::in);
  if(file.is_open())
  {
  while(getline(file, line))
  {
  row.clear();
  
  stringstream str(line);
  
  while(getline(str, word, ','))
  row.push_back(word);
  content.push_back(row);
  }
  }
  else
  cout<<"Could not open the file\n";

  // declaring the arrays.
  float *x, *y, *z, *vx, *vy, *vz;
  float *mass, *h, *hprevious, *rho;
  float *accx, *accy, *accz, *eps;
  float *P, *csnd, *divV, *curlV;
  float *accx_sph, *accy_sph, *accz_sph;
  float *accx_tot, *accy_tot, *accz_tot;
  float *abs_acc_g, *abs_acc_tot, *v_sig, *dh_dt;

  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));
  cudaMallocManaged(&z, N*sizeof(float));

  cudaMallocManaged(&vx, N*sizeof(float));
  cudaMallocManaged(&vy, N*sizeof(float));
  cudaMallocManaged(&vz, N*sizeof(float));

  cudaMallocManaged(&accx, N*sizeof(float));
  cudaMallocManaged(&accy, N*sizeof(float));
  cudaMallocManaged(&accz, N*sizeof(float));

  cudaMallocManaged(&mass, N*sizeof(float));
  cudaMallocManaged(&h, N*sizeof(float));
  cudaMallocManaged(&hprevious, N*sizeof(float));
  cudaMallocManaged(&rho, N*sizeof(float));

  cudaMallocManaged(&eps, N*sizeof(float));
  cudaMallocManaged(&P, N*sizeof(float));
  cudaMallocManaged(&csnd, N*sizeof(float));

  cudaMallocManaged(&divV, N*sizeof(float));
  cudaMallocManaged(&curlV, N*sizeof(float));

  cudaMallocManaged(&accx_sph, N*sizeof(float));
  cudaMallocManaged(&accy_sph, N*sizeof(float));
  cudaMallocManaged(&accz_sph, N*sizeof(float));

  cudaMallocManaged(&accx_tot, N*sizeof(float));
  cudaMallocManaged(&accy_tot, N*sizeof(float));
  cudaMallocManaged(&accz_tot, N*sizeof(float));

  cudaMallocManaged(&abs_acc_g, N*sizeof(float));
  cudaMallocManaged(&abs_acc_tot, N*sizeof(float));
  cudaMallocManaged(&v_sig, N*sizeof(float));
  cudaMallocManaged(&dh_dt, N*sizeof(float));

  // 0  1  2  3   4   5   6  7          8
  // x, y, z, vx, vy, vz, m, hprevious, eps

  // Initialize x, y, and z on the Host.
  for(int i = 0; i < N; i++){
    x[i] = stof(content[i][0]);
    y[i] = stof(content[i][1]);
    z[i] = stof(content[i][2]);

    vx[i] = stof(content[i][3]);
    vy[i] = stof(content[i][4]);
    vz[i] = stof(content[i][5]);

    mass[i] = stof(content[i][6]);
    hprevious[i] = stof(content[i][7]);
    eps[i] = stof(content[i][8]);
    h[i] = 10.0f; // place holder.
    rho[i] = 11.0f; // place holder.
    P[i] = 12.0f; // placeholder.
    csnd[i] = 13.0f; // placeholder.

    divV[i] = 14.0f; // placeholder.
    curlV[i] = 15.0f; // placeholder.

    accx[i] = 0.0f;
    accy[i] = 0.0f;
    accz[i] = 0.0f;

    accx_sph[i] = 0.0f;
    accy_sph[i] = 0.0f;
    accz_sph[i] = 0.0f;

    accx_tot[i] = 0.0f;
    accy_tot[i] = 0.0f;
    accz_tot[i] = 0.0f;

    abs_acc_g[i] = 0.0f;
    abs_acc_tot[i] = 0.0f;
    v_sig[i] = 0.0f;
    dh_dt[i] = 0.0f;
  }

  int blockSize = 256; // number of threads in a block
  int gridSize = (N + blockSize - 1) / blockSize; // Number of blocks in a grid
  
  const int Nngb = 64;
  const int Ndown = Nngb - 5;
  const int Nup = Nngb + 5;
  const float coeff = 0.001; // used for smoothing length.

  const float visc_alpha = 1.0f;

  float mH = 1.6726e-24; // gram
  float kB = 1.3807e-16; // cm2 g s-2 K-1
  float mH2 = muu * mH;

  float kBmH2 = kB/mH2;

  float T_cld = 377.0f;
  float T_ps = 10.0f;
  float T_0 = 10.0f;

  const float my_pi = 3.141592f;

  float t = 0.0f;
  float dt = 2e-4;
  float tEnd = 2.0f;
  float Nt = ceil(tEnd/dt) + 1;

  //-----------------------------------------------
  //-------------- Smoothing Length ---------------
  //-----------------------------------------------

  smoothing_h<<<gridSize, blockSize>>>(x, y, z, h, hprevious,
                                       N, Ndown, Nup, coeff);
  cudaDeviceSynchronize();
  
  //-----------------------------------------------
  //----------------- getDensity ------------------
  //-----------------------------------------------
  
  getDensity<<<gridSize, blockSize>>>(x, y, z, mass,
                                      rho, h, my_pi, N);
  cudaDeviceSynchronize();

  //-----------------------------------------------
  //------------------ getAcc_g -------------------
  //-----------------------------------------------

  acc_g<<<gridSize, blockSize>>>(x, y, z, eps, accx, accy, accz,
                                 mass, G, N);
  cudaDeviceSynchronize();

  //-----------------------------------------------
  //---------------- getPressure ------------------
  //-----------------------------------------------

  //getPressure<<<gridSize, blockSize>>>(P, rho, T_cld, T_ps, T_0, kBmH2,
  //                                     UnitDensity_in_cgs, Unit_P_in_cgs,
  //                                     gammah, N);
  getPressure_Kitsonias<<<gridSize, blockSize>>>(P, rho, UnitDensity_in_cgs, 
                                                 Unit_P_in_cgs, gammah,
                                                 N);
  cudaDeviceSynchronize();

  //-----------------------------------------------
  //----------------- getCsound -------------------
  //-----------------------------------------------

  //getCsound<<<gridSize, blockSize>>>(csnd, rho, T_cld, T_ps, T_0, kBmH2,
  //                                     UnitDensity_in_cgs, unitVelocity,
  //                                     gammah, N);
  getCsound_Kitsonias<<<gridSize, blockSize>>>(csnd, rho, UnitDensity_in_cgs,
                                               unitVelocity, gammah, N);
  cudaDeviceSynchronize();

  //-----------------------------------------------
  //----------------- div_curlV -------------------
  //-----------------------------------------------

  div_curlVel<<<gridSize, blockSize>>>(divV, curlV, x, y, z, vx, vy, vz,
                                       rho, mass, h, my_pi, N);
  cudaDeviceSynchronize();

  //-----------------------------------------------
  //------------------ acc_sph --------------------
  //-----------------------------------------------

  acc_sph<<<gridSize, blockSize>>>(x, y, z, vx, vy, vz, h, csnd, rho,
                                   divV, curlV, mass, P, accx_sph, accy_sph,
                                   accz_sph, my_pi, visc_alpha, N);
  cudaDeviceSynchronize();

  //-----------------------------------------------
  //------------------ acc_tot --------------------
  //-----------------------------------------------
  
  acc_g_sph<<<gridSize, blockSize>>>(accx_tot, accy_tot, accz_tot,
                                     accx, accy, accz,
                                     accx_sph, accy_sph, accz_sph,
                                     N);
  cudaDeviceSynchronize();


  float v_signal, min_h, dt_cfl, max_abs_acc_g, max_abs_acc_tot;
  float dt_f, dt_kin, min_h_dh_dt, dt_dens;
  float dtz[4];

  const float C_CFL = 0.25;


  // ************************************************************** 
  // *********************** MAIN LOOP **************************** 
  // **************************************************************
  
  int counter = 0;

  while(t < tEnd){    

    //****************** velocity evolution *******************
    v_evolve<<<gridSize, blockSize>>>(vx, vy, vz, accx_tot, accy_tot,
                                      accz_tot, dt, N);
    cudaDeviceSynchronize();
    
    //****************** position evolution *******************
    r_evolve<<<gridSize, blockSize>>>(x, y, z, vx, vy, vz, dt, N);
    cudaDeviceSynchronize();

    //****************** Smoothing Length *********************
    smoothing_h<<<gridSize, blockSize>>>(x, y, z, h, hprevious,
                                        N, Ndown, Nup, coeff);
    cudaDeviceSynchronize();

    //****************** updating hprevious ***************
    hprevious_updater<<<gridSize, blockSize>>>(hprevious,
                                               h, N);
    cudaDeviceSynchronize();

    //****************** getDensity ***********************
    getDensity<<<gridSize, blockSize>>>(x, y, z, mass,
                                        rho, h, my_pi, N);
    cudaDeviceSynchronize();

    //****************** getAcc_g *************************
    acc_g<<<gridSize, blockSize>>>(x, y, z, eps, accx, accy, accz,
                                  mass, G, N);
    cudaDeviceSynchronize();

    //****************** getPressure **********************
    //getPressure<<<gridSize, blockSize>>>(P, rho, T_cld, T_ps, T_0, kBmH2,
    //                                    UnitDensity_in_cgs, Unit_P_in_cgs,
    //                                    gammah, N);
    getPressure_Kitsonias<<<gridSize, blockSize>>>(P, rho, UnitDensity_in_cgs, 
                                                 Unit_P_in_cgs, gammah,
                                                 N);
    cudaDeviceSynchronize();

    //****************** getCsound ************************
    //getCsound<<<gridSize, blockSize>>>(csnd, rho, T_cld, T_ps, T_0, kBmH2,
    //                                    UnitDensity_in_cgs, unitVelocity,
    //                                    gammah, N);
    getCsound_Kitsonias<<<gridSize, blockSize>>>(csnd, rho, UnitDensity_in_cgs,
                                               unitVelocity, gammah, N);
    cudaDeviceSynchronize();

    //****************** div_curlV ************************
    div_curlVel<<<gridSize, blockSize>>>(divV, curlV, x, y, z, vx, vy, vz,
                                        rho, mass, h, my_pi, N);
    cudaDeviceSynchronize();

    //****************** acc_sph ************************** 
    acc_sph<<<gridSize, blockSize>>>(x, y, z, vx, vy, vz, h, csnd, rho,
                                    divV, curlV, mass, P, accx_sph, accy_sph,
                                    accz_sph, my_pi, visc_alpha, N);
    cudaDeviceSynchronize();

    //****************** acc_tot **************************
    acc_g_sph<<<gridSize, blockSize>>>(accx_tot, accy_tot, accz_tot,
                                      accx, accy, accz,
                                      accx_sph, accy_sph, accz_sph, N);
    cudaDeviceSynchronize();

    //****************** velocity evolution *******************
    v_evolve<<<gridSize, blockSize>>>(vx, vy, vz, accx_tot, accy_tot,
                                      accz_tot, dt, N);
    cudaDeviceSynchronize();

    counter ++;

    if(!(counter % 50)){
      cout << "t = " << t << endl;
    }

    if(!(counter % 50)){

      ofstream outfile("G-"+ to_string(t) + ".csv");
      
      outfile << "x" << "," << "y" << "," << "z" << "," << "h" << ","
              << "rho" << endl; // this will be the header !

      for(int i = 0; i < N; i++){
        outfile << x[i] << "," << y[i] << "," << z[i] << "," << h[i] << "," 
                << rho[i] << endl;
      }
    }
    
    dt_array<<<gridSize, blockSize>>>(accx, accy, accz, accx_tot,
                                      accy_tot, accz_tot, h, csnd,
                                      abs_acc_g, abs_acc_tot, v_sig,
                                      divV, dh_dt, N);

    v_signal = max_finder(v_sig, N);
    min_h = min_finder(h, N);
    dt_cfl = C_CFL * min_h / v_signal;

    cout << "v_signal, min_h, C_CFL = " << v_signal<<" "<<min_h << " "<< C_CFL<<endl;

    max_abs_acc_g = max_finder(abs_acc_g, N);
    dt_f = sqrt(min_h/max_abs_acc_g);

    max_abs_acc_tot = max_finder(abs_acc_tot, N);
    dt_kin = sqrt(min_h/max_abs_acc_tot);

    min_h_dh_dt = min_finder(dh_dt, N);
    dt_dens = C_CFL * min_h_dh_dt;

    dtz[0] = dt_f; dtz[1] = dt_kin; dtz[2] = dt_cfl; dtz[3] = dt_dens;
    dt = 0.25 * min_finder(dtz, 4);

    cout << "dt_f = " << dt_f << endl;
    cout << "dt_kin = " << dt_kin << endl;
    cout << "dt_cfl = " << dt_cfl << endl;
    cout << "dt_dens = " << dt_dens << endl;

    if(dt > 0.0005){
      dt = 0.0005;
    }

    if(dt < 0.00001){
      dt = 0.00001;
    }

    cout << "Adopted dt = " << dt << endl;
    cout << "current t = " << t << endl;
    cout << "*****************************" << endl;
    cout << endl;

    t += dt;

  }


  cudaFree(x); cudaFree(y); cudaFree(z);
  cudaFree(vx); cudaFree(vy); cudaFree(vz);
  cudaFree(mass); cudaFree(h); cudaFree(hprevious);
  cudaFree(rho); cudaFree(accx); cudaFree(accy); cudaFree(accz);
  cudaFree(P); cudaFree(csnd); cudaFree(divV); cudaFree(curlV);
  cudaFree(accx_sph); cudaFree(accy_sph); cudaFree(accz_sph);
  cudaFree(accx_tot); cudaFree(accy_tot); cudaFree(accz_tot);
  cudaFree(abs_acc_g); cudaFree(abs_acc_tot); cudaFree(v_sig);
  cudaFree(dh_dt);

}

Overwriting hfv_sph_gpu.cu


In [ ]:
%%shell
nvcc hfv_sph_gpu.cu

hfv_sph_gpu.cu(273): warning: variable "T_cld" was declared but never referenced

hfv_sph_gpu.cu(274): warning: variable "T_ps" was declared but never referenced

hfv_sph_gpu.cu(275): warning: variable "T_0" was declared but never referenced



In [ ]:
%%shell
./a.out